In [ ]:
import pathlib

import numpy as np
import pandas as pd

import anndata as ad
import scanpy as sc
import squidpy as sq

import seaborn as sns
import matplotlib.pyplot as plt

import tangram as tg

sc.logging.print_header()
print(f"squidpy=={sq.__version__}")
print(f"tangram=={tg.__version__}")

# Create Squidpy data

In [ ]:
vis_aco = sq.read.visium("../../visium_data/20230807_Visium/B1_Lower/outs")
vis_aco.var_names_make_unique()

In [ ]:
vis_aco

In [ ]:
sc.pp.calculate_qc_metrics(vis_aco, inplace=True)

In [ ]:
sc.pp.normalize_total(vis_aco, inplace=True) #CPM normalizeation
sc.pp.log1p(vis_aco)
sc.pp.highly_variable_genes(vis_aco)
sc.pp.pca(vis_aco)

In [ ]:
sc.pl.pca_variance_ratio(vis_aco, log=True)

In [ ]:
sc.pp.neighbors(vis_aco, n_pcs=15)
sc.tl.umap(vis_aco)
sc.tl.leiden(vis_aco, resolution=0.4)

In [ ]:
meta = pd.read_csv("../../visium_data/20230807_Visium/B1_Lower/outs/Graph-based.csv", index_col=0)

In [ ]:
vis_aco.obs['Graph-based'] = meta

# Analysis

In [ ]:
sc.tl.rank_genes_groups(vis_aco, 'Graph-based')

## Fig S3C : Table genes equivalents (originally by 10x Loupe)

In [ ]:
# THis is 'Fiber'
vis_aco.uns['rank_genes_groups']['names']['Cluster 1'][:30]

In [ ]:
# This is 'lipid'
vis_aco.uns['rank_genes_groups']['names']['Cluster 2'][:30]

In [ ]:
sc.tl.score_genes(vis_aco, gene_list = vis_aco.uns['rank_genes_groups']['names']['Cluster 1'][:30], score_name='Fiber')

In [ ]:
sc.tl.score_genes(vis_aco, gene_list = vis_aco.uns['rank_genes_groups']['names']['Cluster 2'][:30], score_name='Lipid')

## Fig 2G : Spatial Plot equivalents (originally by 10x Loupe)

In [ ]:
sc.pl.spatial(vis_aco, color=['Graph-based','Fiber','Lipid'], alpha_img=0, save='Visium_Score.png')

## Fig S3A: QC Metrics

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
sns.histplot(vis_aco.obs["total_counts"], kde=False, ).figure.savefig("Visium_total_counts.png", dpi=1000, bbox_inches='tight')
plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
sns.histplot(vis_aco.obs["n_genes_by_counts"], kde=False, ).figure.savefig("Visium_gene_counts.png",  dpi=1000, bbox_inches='tight')
plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]

In [ ]:
sc.pl.spatial(vis_aco, color=['total_counts','n_genes_by_counts'], alpha_img=0, save='Visium_QC.png')

## Fig S3B : Gene Expression Spatial Plot equivalents (originally by 10x Loupe)

In [ ]:
sc.pl.spatial(vis_aco, color=['Car3','Lpl','Lum','Loxl2'], alpha_img=0, save='Visium_Score.png')